## The Start of Part-1

### Import Statements

In [2]:
import pandas as pd

### Importing Files

In [6]:
df1 = pd.read_csv('Part1\REQUESTS_FOR_COMMENT.csv')
df2 = pd.read_csv('Part1\USERS.csv')
df3 = pd.read_csv('Part1\WIKIPROJECTS.csv')

# Display the shape of each DataFrame
print(f"Loaded RFC1.csv with shape: {df1.shape}")
print(f"Loaded RFC2.csv with shape: {df2.shape}")
print(f"Loaded RFC3.csv with shape: {df3.shape}")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 5-6: truncated \UXXXXXXXX escape (2183857257.py, line 3)